In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('allSDGtweets.csv', encoding='latin1')

In [14]:
df.head()


,id,created_at,text,lang,#sdg1,#sdg2,#sdg3,#sdg4,#sdg5,#sdg6,...,#sdg10,#sdg11,#sdg12,#sdg13,#sdg14,#sdg15,#sdg16,#sdg17,category,nclasses
0,1.079880e+18,2018-12-31T23:17:02.000Z,"With 300 million people without #EnergyAccess,...",en,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,SDG,1
1,1.079870e+18,2018-12-31T23:00:29.000Z,Happy new year 2019 to #Brussels #SDG11,en,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,SDG,2
2,1.079870e+18,2018-12-31T23:00:00.000Z,Nearly 4 000 new followers joined us on social...,en,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,SDG,2
3,1.079870e+18,2018-12-31T22:35:39.000Z,In 2019 Promoting @Najim_Ali five simple #Pas...,en,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,SDG,1
4,1.079850e+18,2018-12-31T21:30:26.000Z,Please download the #Safecity Mobile app on An...,en,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,SDG,1
